In [1]:
# Imports
import numpy as np
import pandas as pd
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline    

#加载原始数据
raw_data_train = pd.read_csv(r"./dataSets/train.csv", encoding='gbk')
raw_data_test = pd.read_csv(r"./dataSets/test.csv", encoding='gbk')

#缺失值处理
train_data = raw_data_train.fillna(0)
train_feature_data = (train_data.drop(['id','label1','label2'],axis=1))
train_target = train_data.label2
X_train = train_feature_data.values.astype(np.float32)

test_data = raw_data_test.fillna(0)
test_feature_data = (test_data.drop(['id','label1','label2'],axis=1))
test_target = test_data.label2
X_test = test_feature_data.values.astype(np.float32)

X_train, _, y_train, _ = train_test_split(X_train, train_target, test_size=0.0, random_state=123)
_, X_test, _, y_test = train_test_split(X_test, test_target, test_size=0.9999, random_state=123)

In [2]:
#加载原始数据
#raw_data = pd.read_csv(r"./creditGrade_train_data.csv", encoding='gbk')
#data = raw_data.fillna(0)
#features_data = (data.drop(['id','label1','label2'],axis=1))
#5-class
#target = data.label2
#X = features_data.values.astype(np.float32) # 转换数据类型
#X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=123) # 参数test_size设置训练集占比

In [3]:
# Normalization
from sklearn import preprocessing
# l2正则化
X_train = preprocessing.normalize(X_train, norm='l2')
X_test = preprocessing.normalize(X_test, norm='l2')
print(X_train)


[[  2.01437473e-01   0.00000000e+00   2.17514172e-01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  4.90476266e-02   0.00000000e+00   1.65245622e-01 ...,   7.08895456e-03
    1.36673189e-04   2.60942634e-02]
 [  1.11642247e-03   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  6.29723221e-02   0.00000000e+00   1.66234493e-01 ...,   8.97091231e-05
    0.00000000e+00   1.16708950e-04]
 [  1.07931413e-01   0.00000000e+00   1.36101902e-01 ...,   2.62339599e-03
    3.63099825e-05   2.48723361e-03]
 [  9.69535932e-02   0.00000000e+00   1.26169785e-03 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [4]:
#对分类进行one-hot编码
y_tr = one_hot(y_train.astype(np.int64),5)
y_vld = one_hot(y_test.astype(np.int64),5)
print(X_train.shape)
print(X_test.shape)
print("train num:", len(y_tr))
print("val num:", len(y_vld))

(2600, 106)
(100, 106)
train num: 2600
val num: 100


In [ ]:
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
print("tensorflow运行版本：" + tf.__version__)
#设置相关参数
batch_size = 2600  # Batch size
features_num = 106        # Number of steps
learning_rate = 0.00025
epochs = 5000
n_classes = 5
lambda1 = 0.003
log_dir = r'./logs'    # 输出日志保存的路径

graph = tf.Graph()
# Construct placeholders
with graph.as_default():
    in_units = 106
    h1_units = 900
    h2_units = 1024
    #第一层全连接层参数
    W1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W1))
    b1 = tf.Variable(tf.zeros([h1_units]))
    
    #第二层卷积层参数
    Wconv1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev=0.1))
    #tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(Wconv1))
    bconv1 = tf.Variable(tf.constant(0.1, shape=[32]))
    
    #第三层卷积层参数
    Wconv2 = tf.Variable(tf.truncated_normal([3, 3, 32, 32], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(Wconv2))
    bconv2 = tf.Variable(tf.constant(0.1, shape=[32]))
    
    #第四层全连接层参数
    W2 = tf.Variable(tf.truncated_normal([15*15*32, h2_units], stddev=0.1))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W2))
    b2 = tf.Variable(tf.zeros([h2_units]))
    
    #第五层全连接层参数
    W3 = tf.Variable(tf.zeros([h2_units, 5]))
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(W3))
    b3 = tf.Variable(tf.zeros([5]))    
    
    #构造网络
    inputs_ = tf.placeholder(tf.float32, [None, 106], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, 5], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'prob')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

    inputs_ = tf.nn.dropout(inputs_, keep_prob_)
    #第一层全连接层，将维度拓展到784
    hidden1 = tf.nn.relu(tf.matmul(inputs_, W1) + b1)
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob_)
    
    #将数据转换为2维，送入第二层卷积层
    hidden1_drop = tf.reshape(hidden1_drop, [-1, 30, 30, 1])
    h_conv1 = tf.nn.relu(tf.nn.conv2d(hidden1_drop, Wconv1, strides=[1, 1, 1, 1], padding='SAME') + bconv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    h_pool1 = tf.nn.dropout(h_pool1, keep_prob_)
    
    #第三层卷积层
    h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, Wconv2, strides=[1, 1, 1, 1], padding='SAME') + bconv2)
    #h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')    
    #h_pool2 = tf.reshape(h_pool2, [-1, 5*5*64])  #将数据转换为1维
    h_pool2 = tf.nn.dropout(h_conv2, keep_prob_)
    
    h_pool2 = tf.reshape(h_pool2, [-1, 15*15*32])  #将数据转换为1维
    #第四层全连接层，将维度转换为500
    hidden2 = tf.nn.relu(tf.matmul(h_pool2, W2) + b2)
    hidden2_drop = tf.nn.dropout(hidden2, keep_prob_)
    
    #第五层全连接层，维度转换为5，进行5分类
    output_ = tf.matmul(hidden2_drop, W3) + b3
    # Cost function and optimizer
    #二次代价函数，计算预测值与真实值之间的误差代价值-loss，其中第一个参数logits为最后一层输出，第二个为训练目标值即分类值
    #先通过Softmax函数，输出X对应输出每一类的概率大小，其次和真实值进行“交叉熵”，最终，对向量求均值，得到代价loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_, labels=labels_))
    tf.add_to_collection('losses', loss)
    cost = tf.add_n(tf.get_collection('losses'))
    #cost = loss
    
    #梯度下降法，数据量选择AdamOptimizer
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    #correct_pred 返回一个布尔型数组，通过转化为0-1值后来计算准确率
    correct_pred = tf.equal(tf.argmax(output_, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
    #预测值
    pred = tf.argmax(output_, 1, name='cnn')
    print(pred)
    #正确值
    label = tf.argmax(labels_, 1)

tf.summary.scalar("accuracy", accuracy)
tf.summary.scalar('loss_function', cost)

if (os.path.exists(r'./checkpoints') == False):
    !mkdir checkpoints

validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    # summaries合并
    merged = tf.summary.merge_all()    
    # 写到指定的磁盘路径中
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_train, y_tr, batch_size):
            x = x.reshape((batch_size,features_num))
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y,keep_prob_ : 0.45, learning_rate_ : learning_rate}
             # Loss
            summary_str, loss, _ , acc = sess.run([merged, cost, optimizer, accuracy], feed_dict = feed)
         
            train_acc.append(acc)
            train_loss.append(loss)
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
                train_writer.add_summary(summary_str, e);
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                x_v = X_test.reshape(-1, features_num)
                y_v = y_vld
                feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0} 
                summary, loss_v, acc_v, pred_labels = sess.run([merged, cost, accuracy, pred], feed_dict = feed)                    
                val_acc_.append(acc_v)
                val_loss_.append(loss_v)
                
                #for x_v, y_v in get_batches(X_test, y_vld, batch_size):
                    #x_v = x_v.reshape(batch_size, features_num)
                    # Feed
                    #feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}                      
                    # Loss
                    #summary, loss_v, acc_v, pred_labels = sess.run([merged, cost, accuracy, pred], feed_dict = feed)                    
                    #val_acc_.append(acc_v)
                    #val_loss_.append(loss_v)
                    
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                test_writer.add_summary(summary, e);
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            # Iterate 
            iteration += 1    
            
        
#     # 保存二进制模型
#     output_graph_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=['labels'])
#     with tf.gfile.FastGFile(r'D:\py_projects\ML\ML_Demo\financial\creditGrade.pb', mode='wb') as f:
#         f.write(output_graph_def.SerializeToString())

    saver.save(sess,"checkpoints-cnn/creditGrade.ckpt")

    # 保存二进制模型
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=['cnn'])
    with tf.gfile.FastGFile(r'cnn.pb', mode='wb') as f:
        f.write(output_graph_def.SerializeToString())

tensorflow运行版本：1.10.0
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Tensor("cnn:0", shape=(?,), dtype=int64)
Epoch: 4/5000 Iteration: 5 Train loss: 86.459549 Train acc: 0.471154
Epoch: 9/5000 Iteration: 10 Train loss: 84.468910 Train acc: 0.471154
Epoch: 9/5000 Iteration: 10 Validation loss: 84.193436 Validation acc: 0.440000
Epoch: 14/5000 Iteration: 15 Train loss: 82.504745 Train acc: 0.471154
Epoch: 19/5000 Iteration: 20 Train loss: 80.615685 Train acc: 0.471154
Epoch: 19/5000 Iteration: 20 Validation loss: 80.346695 Validation acc: 0.440000
Epoch: 24/5000 Iteration: 25 Train loss: 78.751335 Train acc: 0.471154
Epoch: 29/5000 Iteration: 30 Train loss: 76.938721 Train acc: 0.471154
Epoch: 29/5000 Iteration: 30 Validation loss: 76.685333 Validation acc: 0.440000
Epoch: 34/5000 Iteration: 35 Train loss: 75.169563 Train acc: 0.471154
Epoch: 39

Epoch: 349/5000 Iteration: 350 Train loss: 17.450886 Train acc: 0.542308
Epoch: 349/5000 Iteration: 350 Validation loss: 17.380833 Validation acc: 0.520000
Epoch: 354/5000 Iteration: 355 Train loss: 17.039013 Train acc: 0.554615
Epoch: 359/5000 Iteration: 360 Train loss: 16.638956 Train acc: 0.555769
Epoch: 359/5000 Iteration: 360 Validation loss: 16.582994 Validation acc: 0.530000
Epoch: 364/5000 Iteration: 365 Train loss: 16.249918 Train acc: 0.567692
Epoch: 369/5000 Iteration: 370 Train loss: 15.884161 Train acc: 0.559615
Epoch: 369/5000 Iteration: 370 Validation loss: 15.826724 Validation acc: 0.540000
Epoch: 374/5000 Iteration: 375 Train loss: 15.506008 Train acc: 0.561923
Epoch: 379/5000 Iteration: 380 Train loss: 15.145622 Train acc: 0.566538
Epoch: 379/5000 Iteration: 380 Validation loss: 15.098258 Validation acc: 0.520000
Epoch: 384/5000 Iteration: 385 Train loss: 14.796136 Train acc: 0.579231
Epoch: 389/5000 Iteration: 390 Train loss: 14.447940 Train acc: 0.577308
Epoch: 389/

Epoch: 714/5000 Iteration: 715 Train loss: 2.864194 Train acc: 0.901538
Epoch: 719/5000 Iteration: 720 Train loss: 2.805364 Train acc: 0.901538
Epoch: 719/5000 Iteration: 720 Validation loss: 3.031674 Validation acc: 0.780000
Epoch: 724/5000 Iteration: 725 Train loss: 2.732373 Train acc: 0.906538
Epoch: 729/5000 Iteration: 730 Train loss: 2.655185 Train acc: 0.909231
Epoch: 729/5000 Iteration: 730 Validation loss: 2.879061 Validation acc: 0.740000
Epoch: 734/5000 Iteration: 735 Train loss: 2.598379 Train acc: 0.901923
Epoch: 739/5000 Iteration: 740 Train loss: 2.532416 Train acc: 0.912692
Epoch: 739/5000 Iteration: 740 Validation loss: 2.759552 Validation acc: 0.790000
Epoch: 744/5000 Iteration: 745 Train loss: 2.452259 Train acc: 0.921923
Epoch: 749/5000 Iteration: 750 Train loss: 2.399244 Train acc: 0.914231
Epoch: 749/5000 Iteration: 750 Validation loss: 2.645758 Validation acc: 0.760000
Epoch: 754/5000 Iteration: 755 Train loss: 2.345198 Train acc: 0.913077
Epoch: 759/5000 Iteratio

Epoch: 1079/5000 Iteration: 1080 Train loss: 0.521102 Train acc: 0.969615
Epoch: 1079/5000 Iteration: 1080 Validation loss: 0.865307 Validation acc: 0.820000
Epoch: 1084/5000 Iteration: 1085 Train loss: 0.513003 Train acc: 0.972692
Epoch: 1089/5000 Iteration: 1090 Train loss: 0.510369 Train acc: 0.965385
Epoch: 1089/5000 Iteration: 1090 Validation loss: 0.845408 Validation acc: 0.770000
Epoch: 1094/5000 Iteration: 1095 Train loss: 0.491967 Train acc: 0.971538
Epoch: 1099/5000 Iteration: 1100 Train loss: 0.481612 Train acc: 0.973462
Epoch: 1099/5000 Iteration: 1100 Validation loss: 0.851536 Validation acc: 0.770000
Epoch: 1104/5000 Iteration: 1105 Train loss: 0.474906 Train acc: 0.971154
Epoch: 1109/5000 Iteration: 1110 Train loss: 0.470096 Train acc: 0.973462
Epoch: 1109/5000 Iteration: 1110 Validation loss: 0.805744 Validation acc: 0.850000
Epoch: 1114/5000 Iteration: 1115 Train loss: 0.459699 Train acc: 0.973846
Epoch: 1119/5000 Iteration: 1120 Train loss: 0.450275 Train acc: 0.97076

Epoch: 1434/5000 Iteration: 1435 Train loss: 0.225252 Train acc: 0.983462
Epoch: 1439/5000 Iteration: 1440 Train loss: 0.225427 Train acc: 0.981923
Epoch: 1439/5000 Iteration: 1440 Validation loss: 0.598629 Validation acc: 0.820000
Epoch: 1444/5000 Iteration: 1445 Train loss: 0.227140 Train acc: 0.977692
Epoch: 1449/5000 Iteration: 1450 Train loss: 0.223719 Train acc: 0.985000
Epoch: 1449/5000 Iteration: 1450 Validation loss: 0.596476 Validation acc: 0.790000
Epoch: 1454/5000 Iteration: 1455 Train loss: 0.224513 Train acc: 0.979231
Epoch: 1459/5000 Iteration: 1460 Train loss: 0.222681 Train acc: 0.982308
Epoch: 1459/5000 Iteration: 1460 Validation loss: 0.595693 Validation acc: 0.820000
Epoch: 1464/5000 Iteration: 1465 Train loss: 0.221115 Train acc: 0.980769
Epoch: 1469/5000 Iteration: 1470 Train loss: 0.221769 Train acc: 0.979615
Epoch: 1469/5000 Iteration: 1470 Validation loss: 0.594872 Validation acc: 0.780000
Epoch: 1474/5000 Iteration: 1475 Train loss: 0.226796 Train acc: 0.97769

Epoch: 1789/5000 Iteration: 1790 Train loss: 0.180225 Train acc: 0.985769
Epoch: 1789/5000 Iteration: 1790 Validation loss: 0.589894 Validation acc: 0.790000
Epoch: 1794/5000 Iteration: 1795 Train loss: 0.181288 Train acc: 0.985000
Epoch: 1799/5000 Iteration: 1800 Train loss: 0.182427 Train acc: 0.985385
Epoch: 1799/5000 Iteration: 1800 Validation loss: 0.567924 Validation acc: 0.840000
Epoch: 1804/5000 Iteration: 1805 Train loss: 0.179081 Train acc: 0.986538
Epoch: 1809/5000 Iteration: 1810 Train loss: 0.179874 Train acc: 0.985000
Epoch: 1809/5000 Iteration: 1810 Validation loss: 0.570712 Validation acc: 0.770000
Epoch: 1814/5000 Iteration: 1815 Train loss: 0.176064 Train acc: 0.986154
Epoch: 1819/5000 Iteration: 1820 Train loss: 0.178121 Train acc: 0.985000
Epoch: 1819/5000 Iteration: 1820 Validation loss: 0.584180 Validation acc: 0.800000
Epoch: 1824/5000 Iteration: 1825 Train loss: 0.182225 Train acc: 0.982692
Epoch: 1829/5000 Iteration: 1830 Train loss: 0.182529 Train acc: 0.98115

In [ ]:
t = np.arange(iteration-1)
#print(np.array(train_loss))
plt.figure(figsize = (9,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (9,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
test_acc = []
pred_labels = []
label = []
with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
    x_v = X_test.reshape(-1, features_num)
    y_v = y_vld
    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0} 
    batch_acc = sess.run(accuracy, feed_dict=feed)
    preds = sess.run(pred, feed_dict=feed)
    labels = sess.run(label, feed_dict=feed)
    #max_index = np.argmax(prediction)
    #print(max_index)
    test_acc.append(batch_acc)
    pred_labels.append(preds)
    label.append(labels)
    
    #for x_t, y_t in get_batches(X_test, y_vld, batch_size):
        #x_t = x_t.reshape((batch_size, features_num))
        #feed = {inputs_: x_t,
        #        labels_: y_t,
        #        keep_prob_: 1}
       
        #batch_acc = sess.run(accuracy, feed_dict=feed)
        #preds = sess.run(pred, feed_dict=feed)
        #labels = sess.run(label, feed_dict=feed)
        #test_acc.append(batch_acc)
        #pred_labels.append(preds)
        #label.append(labels)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))
    print("pred value", pred_labels)